In [ ]:
#create mark down cell explaining what's up

In [24]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, export_graphviz

readmit = pd.read_csv('readmit_for_trees.csv')

In [21]:
features = list(readmit.columns[:47])

In [26]:
Y = readmit['readmit30']
X = readmit[features]
dt = DecisionTreeClassifier(min_samples_split=20, random_state=99)
dt.fit(X, Y)

ValueError: could not convert string to float: 'Yes'